In [111]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

In [112]:
import warnings
warnings.filterwarnings('ignore')

### By country

In [3]:
df = pd.read_excel(r'D:/data/informal_ml_community/Olympics 2018.xlsx', sheet_name = 'Olympics')

In [4]:
df = df[['Sort', 'Year', 'Sport', 'Gender', 'Event', 'Athlete', 'Age', 'Result',
       'Medal', 'City', 'Country', 'Team', 'Season', 'Team Medals', 'OR', 'WR',
       'Notes']]

In [5]:
df2 = df[(df.Season == 'Summer')][['Year','Team','Medal','Team Medals']]

In [6]:
df2 = df2[df2.Medal.isin(['GOLD','SILVER','BRONZE'])]

In [7]:
df3 = df2.groupby(['Year','Team','Medal']).sum()
df3 = pd.DataFrame(df3.to_records())

In [8]:
df4 = df3.pivot(index=['Year','Team'], columns='Medal', values='Team Medals')
df4 = pd.DataFrame(df4.to_records())

In [9]:
df4

,Year,Team,BRONZE,GOLD,SILVER
0,1896,Australia,1.0,NaN,NaN
1,1896,Austria,2.0,4.0,1.0
2,1896,Denmark,3.0,1.0,2.0
3,1896,Egypt,NaN,NaN,1.0
4,1896,France,NaN,1.0,NaN
...,...,...,...,...,...
1268,2016,United Arab Emirates,1.0,NaN,NaN
1269,2016,United States,36.0,46.0,37.0
1270,2016,Uzbekistan,7.0,4.0,2.0
1271,2016,Venezuela,2.0,NaN,1.0


In [ ]:
df3

### By athletes

In [2]:
athlete_file = r'D:/data/informal_ml_community/athlete_events.txt'
df = pd.read_csv(athlete_file,index_col=0)

In [3]:
df.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
ID,,,,,,,,,,,,,,
1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


#### Case 1) Country basis

In [4]:
df_noc = df[(df.Season == 'Summer') & (df.Year > 1995)]
df_noc = df_noc[['Name','NOC','Year','City','Medal']]

In [5]:
df_noc.loc[:,'HasMedal'] = df_noc.Medal.apply(lambda x: 0 if isinstance(x, float) else 1)

In [6]:
df_noc = pd.DataFrame(df_noc.groupby(['NOC','Year', 'City']).sum().to_records())

In [7]:
df_noc[(df_noc.NOC == 'KOR')]

,NOC,Year,City,HasMedal
606,KOR,1996,Atlanta,66
607,KOR,2000,Sydney,73
608,KOR,2004,Athina,52
609,KOR,2008,Beijing,78
610,KOR,2012,London,61
611,KOR,2016,Rio de Janeiro,26


In [8]:
list(df_noc.City.drop_duplicates())

['Atlanta', 'Athina', 'Beijing', 'London', 'Rio de Janeiro', 'Sydney']

In [9]:
city_cc_mapper = {'Atlanta': 'USA', 'Athina':'GRC', 'Beijing':'CHN', 'London':'GBR', 'Rio de Janeiro':'BRA', 'Sydney':'AUS'}

In [10]:
df_noc.loc[:,'Host'] = df_noc.City.apply(lambda x: city_cc_mapper[x])

In [11]:
df_noc

,NOC,Year,City,HasMedal,Host
0,AFG,1996,Atlanta,0,USA
1,AFG,2004,Athina,0,GRC
2,AFG,2008,Beijing,1,CHN
3,AFG,2012,London,1,GBR
4,AFG,2016,Rio de Janeiro,0,BRA
...,...,...,...,...,...
1209,ZIM,2000,Sydney,0,AUS
1210,ZIM,2004,Athina,3,GRC
1211,ZIM,2008,Beijing,4,CHN
1212,ZIM,2012,London,0,GBR


In [12]:
df_noc.columns = ['guest', 'year', 'city', 'medal', 'host']
df_noc = df_noc[['year','host','guest','medal']]

In [13]:
df_noc[df_noc.year == 2016].sort_values('medal',ascending=False)

,year,host,guest,medal
1165,2016,BRA,USA,264
436,2016,BRA,GER,159
412,2016,BRA,GBR,145
941,2016,BRA,RUS,115
235,2016,BRA,CHN,113
...,...,...,...,...
565,2016,BRA,IVB,0
601,2016,BRA,KGZ,0
605,2016,BRA,KIR,0
14,2016,BRA,ALB,0


In [14]:
df_noc.loc[:,'host_advantage'] = df_noc.apply(lambda x: 1 if x.host == x.guest else 0,axis=1)

In [15]:
df_noc[df_noc.host_advantage == 1]

,year,host,guest,medal,host_advantage
64,2000,AUS,AUS,183,1
164,2016,BRA,BRA,50,1
233,2008,CHN,CHN,184,1
411,2012,GBR,GBR,126,1
1160,1996,USA,USA,259,1


In [16]:
df_noc[df_noc.guest == 'BRA']

,year,host,guest,medal,host_advantage
159,1996,USA,BRA,63,0
160,2000,AUS,BRA,48,0
161,2004,GRC,BRA,40,0
162,2008,CHN,BRA,78,0
163,2012,GBR,BRA,59,0
164,2016,BRA,BRA,50,1


In [ ]:
#!pip install scikit-learn==0.23.2 scikit-optimize

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score, mean_squared_error

In [18]:
df_noc_test = df_noc[df_noc.year == 2016].drop('year',axis=1)
df_noc_train = df_noc[~df_noc.index.isin(df_noc_test.index)].drop('year', axis=1)

In [19]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3457)
X_train = df_noc_train.drop('medal', axis=1)
y_train = df_noc_train['medal']
X_test = df_noc_test.drop('medal', axis=1)
y_test = df_noc_test['medal']

In [108]:
regressors = [
    LinearRegression(),
    SGDRegressor(),
    SVR(),
    KNeighborsRegressor(),
    RandomForestRegressor(),
    MLPRegressor()
]

In [109]:
for regressor in regressors:
    pipeline = Pipeline(steps=[('preprocessor',
                    ColumnTransformer(transformers=[('categorical',
                                                     Pipeline(steps = [('encoder', OneHotEncoder(handle_unknown='ignore'))]),
                                                     ['host','guest','host_advantage'])
                                                   ]
                                     )
                    ),
                    ('regressor', regressor)
                   ]
            )
    ml_model = pipeline.fit(X_train, y_train)
    predictions = ml_model.predict(X_test)
    print(f'Model: {regressor}, RMSE: {round((np.sqrt(mean_squared_error(y_test,predictions))),2)}, r2 score: {round(r2_score(y_test, predictions),2)}')

Model: LinearRegression(), RMSE: 11.62, r2 score: 0.84
Model: SGDRegressor(), RMSE: 18.21, r2 score: 0.6
Model: SVR(), RMSE: 29.87, r2 score: -0.06


d:\workspace\env\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1225: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


Model: KNeighborsRegressor(), RMSE: 13.29, r2 score: 0.79
Model: RandomForestRegressor(), RMSE: 12.05, r2 score: 0.83
Model: MLPRegressor(), RMSE: 20.45, r2 score: 0.5


d:\workspace\env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [72]:
from sklearn import metrics

model = Pipeline([('scaler', StandardScaler()), ('model', MLPRegressor())])

search_params = {
    Categorical(['tanh','relu'],name='activation'),
    Integer(1,4,name='n_hidden_layer'),
    Integer(200,2000,name='n_neurons_per_layer')
}

searchcv = BayesSearchCV(
    estimator=model,
    search_spaces=search_params,
    n_iter=100,
    n_jobs=-1,
    cv=5,
    refit=True,
    error_score=0,
    scoring=metrics.make_scorer(metrics.balanced_accuracy_score),
    return_train_score=True,
    verbose=1
)
searchcv.fit(X_train, y_train)

TypeError: unhashable type: 'Categorical'

In [ ]:
# evaluate model
m_scores = cross_val_score(model, pd.DataFrame(bar,columns= ['host','guest','host_advantage']), y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(m_scores), std(m_scores)))

OrdinalEncoder()

In [92]:
from time import time
from sklearn.metrics import average_precision_score
from sklearn.metrics import make_scorer

# Model selection
from sklearn.model_selection import StratifiedKFold

In [78]:
avg_prec = make_scorer(average_precision_score, greater_is_better=True, needs_proba=True)

In [79]:
# Reporting util for different optimizers
def report_perf(optimizer, X, y, title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    best_score = optimizer.best_score_
    best_score_std = optimizer.cv_results_['std_test_score'][optimizer.best_index_]
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [99]:
clf = RandomForestRegressor()

In [84]:
# Setting a 5-fold stratified cross-validation (note: shuffle=True)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [101]:
search_spaces = {
                 "subsample": Real(0.5, 1.0),
                 "min_samples_split": Integer(2, 10),
                 "min_samples_leaf": Integer(1, 10),
                 "max_features": Categorical(categories=['sqrt', 'log2', None])}

for baseEstimator in ['GP', 'RF', 'ET', 'GBRT']:
    opt = BayesSearchCV(clf,
                        search_spaces,
                        scoring=avg_prec,
                        cv=skf,
                        n_iter=40,
                        n_jobs=-1,
                        return_train_score=False,
                        optimizer_kwargs={'base_estimator': baseEstimator},
                        random_state=4)
    
    best_params = report_perf(opt, X, y_train,'BayesSearchCV_'+baseEstimator)

d:\workspace\env\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


ValueError: Invalid parameter subsample for estimator RandomForestRegressor(max_features=None, min_samples_leaf=8,
                      min_samples_split=4). Check the list of available parameters with `estimator.get_params().keys()`.

In [88]:
foo = OrdinalEncoder()
foo.fit(X_train)
bar = foo.transform(X_train)
X= pd.DataFrame(bar,columns= ['host','guest','host_advantage'])
y = y_train

In [87]:
y_train

0       0
1       0
2       1
3       1
5       0
       ..
1208    0
1209    0
1210    3
1211    4
1212    0
Name: medal, Length: 1007, dtype: int64

In [98]:
RandomForestRegressor().get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])